# TCN model

To get simulation data run `./run_sim.sh [small, medium, large, test or custom]` from the `leak_model` directory

Then run `gunzip [file_path]` in your terminal to turn the gz file output from the sim into a csv

## Loading data

In [ ]:
import os
from datetime import datetime
import pandas as pd
import numpy as np
import math
import qexpy as q
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
from darts import TimeSeries
from darts.dataprocessing.transformers import MissingValuesFiller
from darts.dataprocessing.transformers import Scaler
from darts.models import TCNModel


In [ ]:
df = pd.read_csv('../sim_data/synthetic_water_data_minute_1000.csv')
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

## Leak df

In [ ]:
leak_df = df.copy()

In [ ]:
leak_df.isnull().sum()

In [ ]:
nan_cols = ['leak_category','leak_branch','leak_pipe']
leak_df[nan_cols] = leak_df[nan_cols].fillna(value='none')
leak_df.head()

In [ ]:
leak_df.isnull().sum()

## Calculating velocity and flow erros

In [ ]:
def calculate_errors(df):
    theta_deg = 60
    theta_cos = np.cos(np.radians(theta_deg))
    epsilon = 1e-10

    # Extract needed fields as arrays
    L = df['l_path_m'].values
    t_u = df['upstream_transit_time_s'].values
    t_d = df['downstream_transit_time_s'].values
    V_actual = df['velocity_m_per_s'].values
    Q_actual = df['flow_m3_s'].values
    id_m = df['id_mm'].values / 1000  # mm to m

    # Step 1: Estimate velocity
    V_est = (L / (2 * theta_cos)) * ((1 / t_d) - (1 / t_u))

    # Step 2: Area of circular pipe
    A = (np.pi/4) * (id_m ** 2)

    # Step 3: Estimate flow
    Q_est = V_est * A

    # Step 4: Error metrics
    velocity_error = np.abs(V_actual - V_est) / np.maximum(np.abs(V_actual), epsilon)
    flow_rate_error = np.abs(Q_actual - Q_est) / np.maximum(np.abs(Q_actual), epsilon)

    # Step 5: Add to DataFrame
    df['V_est'] = V_est
    df['Q_est'] = Q_est
    df['velocity_error'] = velocity_error
    df['flow_rate_error'] = flow_rate_error
    df['velocity_error_pass'] = velocity_error <= 0.05
    df['flow_rate_error_pass'] = flow_rate_error <= 0.05

    return df

In [ ]:
# Apply error calculations to the dataframe
calculate_errors(leak_df)

In [ ]:
leak_df.columns.tolist()

In [ ]:
leak_df[leak_df['pipe_burst_leak'] == True]['house_id'].nunique()

In [ ]:
leak_df[leak_df['pipe_burst_leak'] == True]['leak_pipe'].unique()

In [ ]:
houses = [100]

# Create the plot
plt.figure(figsize=(15, 8))

# Plot flow data for each random house
for house_id in houses:
    house_data = leak_df[leak_df['house_id'] == house_id].copy()
    house_data['timestamp'] = pd.to_datetime(house_data['timestamp'])
    house_data = house_data.sort_values('timestamp')
    
    plt.plot(house_data['timestamp'], house_data['flow_m3_s'], 
             label=f'House {house_id}', alpha=0.7, linewidth=1)

plt.xlabel('Time')
plt.ylabel('Flow Rate (m³/s)')
plt.title('Flow Rate Over Time a house')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Print the selected house IDs
print(f"Selected house IDs: {sorted(houses)}")

# TCN development

In [ ]:
feat_eng_df = leak_df.copy()

In [ ]:
# Defining features and targets
NUMERICAL_FEATURES_v1 = [
    'velocity_m_per_s', 'flow_m3_s', 'upstream_transit_time_s', 
    'downstream_transit_time_s', 'delta_t_ns', 'pipe_width_in',
    'od_mm', 'wall_mm', 'id_mm', 'c_est_m_per_s', 'temp_est_c'
]

CATEGORICAL_FEATURES_v1 = [
    'pipe_material', 'leak_type', 'leak_category', 'leak_pipe'
]

TARGET_COLUMNS = ['pipe_burst_leak', 'leak_branch']

In [ ]:
missing_numerical = [col for col in NUMERICAL_FEATURES_v1 if col not in feat_eng_df.columns]
missing_categorical = [col for col in CATEGORICAL_FEATURES_v1 if col not in feat_eng_df.columns]
missing_targets = [col for col in TARGET_COLUMNS if col not in feat_eng_df.columns]

if missing_numerical:
    print(f"Missing numerical features: {missing_numerical}")
if missing_categorical:
    print(f"Missing categorical features: {missing_categorical}")
if missing_targets:
    print(f"Missing target columns: {missing_targets}")

### One-hot encoding

In [ ]:
def create_feature_encodings(df):
    """Create one-hot encodings for categorical features"""
    df_encoded = df.copy()
    
    # One-hot encode categorical features
    for cat_feature in CATEGORICAL_FEATURES_v1:
        if cat_feature in df_encoded.columns:
            # Get unique values
            unique_vals = df_encoded[cat_feature].unique()
            print(f"{cat_feature} unique values: {unique_vals}")
            
            # Create one-hot encoding
            encoded_cols = pd.get_dummies(df_encoded[cat_feature], 
                                        prefix=f'{cat_feature}_onehot', 
                                        prefix_sep='_')
            
            # Add encoded columns to dataframe
            df_encoded = pd.concat([df_encoded, encoded_cols], axis=1)
            
            print(f"Created {len(encoded_cols.columns)} columns for {cat_feature}")
    
    return df_encoded

In [ ]:
feat_eng_df = create_feature_encodings(feat_eng_df)
feat_eng_df.head()

In [ ]:
def prepare_categorical_targets(df, target_cols):
    """Prepare both binary and categorical targets"""
    df_targets = df.copy()
    
    for target_col in target_cols:
        if target_col in df_targets.columns:
            unique_vals = df_targets[target_col].unique()
            
            if target_col == 'pipe_burst_leak':
                # Binary target - keep as is (boolean/binary)
                print(f"{target_col}: Binary target, keeping as boolean")
                
            elif target_col == 'leak_branch':
                # Categorical target - we have options:
                print(f"{target_col} unique values: {unique_vals}")
                
                # One-hot encode for multi-output
                target_encoded = pd.get_dummies(df_targets[target_col], 
                                              prefix=f'{target_col}_onehot', 
                                              prefix_sep='_')
                df_targets = pd.concat([df_targets, target_encoded], axis=1)
                print(f"One-hot encoded {target_col}: {target_encoded.columns.tolist()}")
    
    return df_targets

In [ ]:
feat_eng_df = prepare_categorical_targets(feat_eng_df, TARGET_COLUMNS)
feat_eng_df.head()

In [ ]:
feat_eng_df.columns.tolist()


### Forward shifting targets

In [ ]:
def create_supervised_learning_targets(df, forecast_horizon: int = 24): # 24 15-min sections = 6 hours
    """
    Create supervised learning targets for the TCN model. Currently, this 
    creates a prediction if there will be any leaks in the next 24 hours.
    """
    
    df_with_targets = df.copy()
    df_with_targets["timestamp"] = pd.to_datetime(df_with_targets["timestamp"])
    df_with_targets = df_with_targets.sort_values(["house_id", "timestamp"])
    
    future_targets = []
    
    for house_id in df_with_targets["house_id"].unique():
        house_data = df_with_targets[df_with_targets["house_id"] == house_id].copy()
        
        # Rolling window - predict if leak occurs anywhere in next 24 hours
        house_data["pipe_burst_leak_next_24h"] = (
            house_data["pipe_burst_leak"]
            .rolling(window=forecast_horizon, min_periods=1)
            .max()
            .shift(-forecast_horizon + 1)
            .fillna(False)
            .astype(bool)
        )
        
        onehot_cols = [col for col in house_data.columns if col.startswith("leak_branch_onehot_")]
        
        for col in onehot_cols:
            # Only mark branch as True if leak happens on that specific branch
            leak_on_this_branch = (house_data[col] & house_data["pipe_burst_leak"])
            
            house_data[f"{col}_next_24h"] = (
                leak_on_this_branch
                .rolling(window=forecast_horizon, min_periods=1)
                .max()
                .shift(-forecast_horizon + 1)
                .fillna(False)
                .astype(bool)
            )
        
        # Handle "none" case - True only when no leak predicted at all
        none_flag = f"leak_branch_onehot_none_next_24h"
        if none_flag in house_data.columns:
            house_data[none_flag] = (
                house_data["pipe_burst_leak_next_24h"] == False
            )
        
        future_targets.append(house_data)
    
    return pd.concat(future_targets, axis=0, ignore_index=True).dropna(
        subset=["pipe_burst_leak_next_24h"]
    )

In [ ]:
feat_eng_df = create_supervised_learning_targets(feat_eng_df)
feat_eng_df.head()

In [ ]:
feat_eng_df[feat_eng_df['pipe_burst_leak_next_24h'] == True]

In [ ]:
feat_eng_df[(feat_eng_df['pipe_burst_leak_next_24h'] == True) & (feat_eng_df['leak_branch_onehot_POWDER_ROOM_BRANCH_next_24h'] == True)]

In [ ]:
leak_branch_24h_cols = [col for col in feat_eng_df.columns if col.startswith('leak_branch_onehot_') and col.endswith('_next_24h')]
feat_eng_df[(feat_eng_df['pipe_burst_leak_next_24h']) & (feat_eng_df[leak_branch_24h_cols].eq(False).any(axis=1))]

### Final feature summary

In [ ]:
final_df = feat_eng_df.copy()

In [ ]:
def summarize_tcn_features_final(df_final):
    """Final summary with target options (updated for _next_24h convention)"""
    
    metadata_cols = ['timestamp', 'house_id'] + CATEGORICAL_FEATURES_v1 + TARGET_COLUMNS
    
    feature_cols = [
        col for col in df_final.columns
        if col not in metadata_cols
        and not col.endswith(('_next_24h', 'pressure_psi'))
        and not col.startswith('leak_branch_onehot_')
        and not col.startswith(('V_est', 'Q_est'))
        and not col.endswith(('_error', '_pass'))
    ]

    onehot_targets = [c for c in df_final.columns
                     if c.startswith('leak_branch_onehot_') and c.endswith('_next_24h')]

    summary = {
        'rows': len(df_final),
        'num_features': len(feature_cols),
        'feature_columns': feature_cols,
        'binary_target': 'pipe_burst_leak_next_24h',
        'categorical_targets': onehot_targets,
        'num_categorical_targets': len(onehot_targets)
    }
    
    print("=== FINAL TCN DATA SUMMARY ===")
    print(f"Rows: {len(df_final):,}")
    print(f"Features: {len(feature_cols)}")
    print("Target options:")
    print("  - Binary: pipe_burst_leak_next_24h")
    print(f"  - Categorical (one-hot): {len(onehot_targets)} columns")
    
    return summary

In [ ]:
summarize_tcn_features_final(final_df)

### Target distribution analysis

In [ ]:
# Target Distribution Validation
print("="*60)
print("TARGET DISTRIBUTION VALIDATION")
print("="*60)

# 1. OVERALL LEAK STATISTICS
print("\n1. OVERALL LEAK STATISTICS")
print("-"*40)

total_samples = len(final_df)
leak_samples = final_df['pipe_burst_leak'].sum()
leak_rate = leak_samples / total_samples

print(f"Total samples: {total_samples:,}")
print(f"Leak samples: {leak_samples:,}")
print(f"No-leak samples: {total_samples - leak_samples:,}")
print(f"Overall leak rate: {leak_rate:.4f} ({leak_rate*100:.2f}%)")
print(f"Class imbalance ratio: {(total_samples - leak_samples) / leak_samples:.1f}:1 (no-leak:leak)")

# 2. PER-HOUSE LEAK DISTRIBUTION
print("\n2. PER-HOUSE LEAK DISTRIBUTION")
print("-"*40)

house_leak_stats = []
for house_id in sorted(final_df['house_id'].unique()):
    house_data = final_df[final_df['house_id'] == house_id]
    house_total = len(house_data)
    house_leaks = house_data['pipe_burst_leak'].sum()
    house_leak_rate = house_leaks / house_total if house_total > 0 else 0
    
    house_leak_stats.append({
        'house_id': house_id,
        'total_samples': house_total,
        'leak_samples': house_leaks,
        'leak_rate': house_leak_rate
    })

house_stats_df = pd.DataFrame(house_leak_stats)

print(f"Houses with leaks: {(house_stats_df['leak_samples'] > 0).sum()}")
print(f"Houses without leaks: {(house_stats_df['leak_samples'] == 0).sum()}")
print(f"Average leak rate per house: {house_stats_df['leak_rate'].mean():.4f}")
print(f"Std leak rate per house: {house_stats_df['leak_rate'].std():.4f}")
print(f"Min leak rate: {house_stats_df['leak_rate'].min():.4f}")
print(f"Max leak rate: {house_stats_df['leak_rate'].max():.4f}")

# 3. LEAK RATE THRESHOLD ANALYSIS (20%)
print("\n3. LEAK RATE THRESHOLD ANALYSIS (20%)")
print("-"*40)

threshold = 0.15  # 20%
houses_over_threshold = house_stats_df[house_stats_df['leak_rate'] > threshold]
houses_at_or_below_threshold = house_stats_df[house_stats_df['leak_rate'] <= threshold]

print(f"Houses with leak rate > 15%: {len(houses_over_threshold)}")
print(f"Houses with leak rate ≤ 15%: {len(houses_at_or_below_threshold)}")
print(f"Percentage of houses > 15%: {len(houses_over_threshold) / len(house_stats_df) * 100:.2f}%")
print(f"Percentage of houses ≤ 15%: {len(houses_at_or_below_threshold) / len(house_stats_df) * 100:.2f}%")

# Show houses with highest leak rates
print(f"\nTop 10 houses by leak rate:")
top_leak_houses = house_stats_df.nlargest(10, 'leak_rate')
for _, row in top_leak_houses.iterrows():
    print(f"House {row['house_id']}: {row['leak_samples']}/{row['total_samples']} = {row['leak_rate']:.4f}")

# Show houses over 20% threshold
if len(houses_over_threshold) > 0:
    print(f"\nHouses with leak rate > 15%:")
    for _, row in houses_over_threshold.iterrows():
        print(f"House {row['house_id']}: {row['leak_samples']}/{row['total_samples']} = {row['leak_rate']:.4f}")
else:
    print(f"\nNo houses have leak rate > 15%")

# 4. CREATE FILTERED DATAFRAME
print("\n4. CREATE FILTERED DATAFRAME")
print("-"*40)

# Get house IDs to keep (those with leak rate <= threshold)
houses_to_keep = houses_at_or_below_threshold['house_id'].tolist()

# Create filtered DataFrame
ff_df = final_df[final_df['house_id'].isin(houses_to_keep)].copy()

print(f"Original dataset size: {len(final_df):,} rows")
print(f"Filtered dataset size: {len(ff_df):,} rows")
print(f"Removed {len(final_df) - len(ff_df):,} rows")
print(f"Removed {len(houses_over_threshold)} houses with leak rate > {threshold*100}%")

# Verify the filtering worked correctly
print(f"\nVerification:")
print(f"Remaining houses: {ff_df['house_id'].nunique()}")
print(f"Removed houses: {len(houses_over_threshold)}")

# Check leak rate in filtered dataset
filtered_total = len(ff_df)
filtered_leaks = ff_df['pipe_burst_leak'].sum()
filtered_leak_rate = filtered_leaks / filtered_total if filtered_total > 0 else 0

print(f"Filtered dataset leak rate: {filtered_leak_rate:.4f} ({filtered_leak_rate*100:.2f}%)")
print(f"Original dataset leak rate: {leak_rate:.4f} ({leak_rate*100:.2f}%)")

### Features and targets

In [ ]:
FEATURES = [
    'velocity_m_per_s', 'flow_m3_s', 'upstream_transit_time_s',
    'downstream_transit_time_s', 'delta_t_ns', 'pipe_width_in', 'od_mm',
    'wall_mm', 'id_mm', 'l_path_m', 'c_est_m_per_s', 'temp_est_c',
    'n_traverses', 'theta_deg', 'pipe_material_onehot_Copper',
    'pipe_material_onehot_PEX', 'leak_type_onehot_burst_freeze',
    'leak_type_onehot_burst_pressure', 'leak_type_onehot_gradual',
    'leak_type_onehot_micro', 'leak_type_onehot_none',
    'leak_category_onehot_dish', 'leak_category_onehot_faucet',
    'leak_category_onehot_laundry', 'leak_category_onehot_none',
    'leak_category_onehot_shower', 'leak_category_onehot_toilet',
    'leak_category_onehot_unknown', 'leak_pipe_onehot_P_DISHWASHER',
    'leak_pipe_onehot_P_ENS_LAV', 'leak_pipe_onehot_P_ENS_SHWR',
    'leak_pipe_onehot_P_ENS_WC', 'leak_pipe_onehot_P_FAM_LAV',
    'leak_pipe_onehot_P_FAM_TUB', 'leak_pipe_onehot_P_FAM_WC',
    'leak_pipe_onehot_P_HOSE_BACK', 'leak_pipe_onehot_P_HOSE_FRONT',
    'leak_pipe_onehot_P_KITCHEN_BRANCH', 'leak_pipe_onehot_P_KITCHEN_SINK',
    'leak_pipe_onehot_P_LAUNDRY', 'leak_pipe_onehot_P_MAIN_1',
    'leak_pipe_onehot_P_MAIN_2', 'leak_pipe_onehot_P_POWDER_BRANCH',
    'leak_pipe_onehot_P_POWDER_LAV', 'leak_pipe_onehot_P_POWDER_WC',
    'leak_pipe_onehot_P_UPPER_BRANCH', 'leak_pipe_onehot_P_WATER_HEATER',
    'leak_pipe_onehot_none'
]

In [ ]:
TARGETS = [
    'pipe_burst_leak_next_24h',
    'leak_branch_onehot_KITCHEN_BRANCH_next_24h',
    'leak_branch_onehot_MAIN_TRUNK_1_next_24h',
    'leak_branch_onehot_MAIN_TRUNK_2_next_24h',
    'leak_branch_onehot_POWDER_ROOM_BRANCH_next_24h',
    'leak_branch_onehot_UPPER_FLOOR_BRANCH_next_24h',
    'leak_branch_onehot_none_next_24h',
    'leak_branch_onehot_unknown_next_24h'
    ]

### Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
import joblib

folder_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

def scale_features_simple(df):
    """Simple sklearn-based scaling for numerical features"""
    df_scaled = df.copy()

    # Separate numerical and one-hot features
    numerical_features = [
        'velocity_m_per_s', 'flow_m3_s', 'flow_gpm', 'upstream_transit_time_s',
        'downstream_transit_time_s', 'delta_t_ns', 'pipe_width_in', 'od_mm',
        'wall_mm', 'id_mm', 'l_path_m', 'c_est_m_per_s', 'temp_est_c',
        'n_traverses', 'theta_deg'
    ]

    # Fit scaler on numerical features
    scaler = StandardScaler()
    df_scaled[numerical_features] = scaler.fit_transform(df_scaled[numerical_features])

    # Create the directory if it doesn't exist
    scaler_path = f'weights/{folder_timestamp}/feature_scaler.pkl'
    os.makedirs(os.path.dirname(scaler_path), exist_ok=True)
    
    # Save the scaler
    joblib.dump(scaler, scaler_path)
    print(f"Scaler saved to {scaler_path}")

    # Show scaling stats
    for col in numerical_features[:5]:
        print(f"{col}: mean={df_scaled[col].mean():.3f}, std={df_scaled[col].std():.3f}")

    return df_scaled, scaler

In [ ]:
ff_scaled, feature_scaler = scale_features_simple(ff_df)

In [ ]:
ff_scaled.head()

### Darts timeseries objects

In [ ]:
dupes_mask = ff_scaled.duplicated(subset=['house_id', 'timestamp'], keep=False)

# How many duplicates overall
print(f"Total duplicate rows: {dupes_mask.sum():,}")

# Count duplicates per house (only the problematic ones)
dupes_per_house = (
    ff_scaled.loc[dupes_mask, ['house_id', 'timestamp']]
    .groupby('house_id')
    .size()
    .sort_values(ascending=False)
)

print("\nTop houses with duplicate timestamps:")
print(dupes_per_house.head(10))

# Inspect a specific house to verify:
sample_house = dupes_per_house.index[0] if not dupes_per_house.empty else None
if sample_house is not None:
    display(
        ff_scaled[ff_scaled['house_id'] == sample_house]
        .loc[lambda d: d.duplicated(subset='timestamp', keep=False)]
        .sort_values('timestamp')
        .head(20)
    )

In [ ]:
dup_df = ff_scaled.copy()
before = len(dup_df)

dup_df = (
    dup_df
    .sort_values(['house_id', 'timestamp'])
    .drop_duplicates(subset=['house_id', 'timestamp'],
                     keep='first')
    .reset_index(drop=True)
)

after = len(dup_df)
print(f"Removed {before - after:,} duplicate rows "
      f"({before:,} → {after:,})")

In [ ]:
df_ts = dup_df.copy()
drop_cols = FEATURES + TARGETS
df_ts[df_ts[drop_cols].isna().any(axis=1)]

In [ ]:
# Drop any remaining NaNs in features/targets only
df_ts = df_ts.dropna(subset=drop_cols)

# Sort by house_id and timestamp
df_ts['timestamp'] = pd.to_datetime(df_ts['timestamp'])
df_ts = df_ts.sort_values(['house_id', 'timestamp'])

# Convert targets and features to float32 for probability modelling
df_ts[TARGETS] = df_ts[TARGETS].astype("float32")
df_ts[FEATURES] = df_ts[FEATURES].astype("float32") 

In [ ]:
df_ts.shape

In [ ]:
# Building a series per house
house_feature_series   = {} # id -> TimeSeries of covariates
house_target_series    = {} # id -> TimeSeries of targetsv

In [ ]:
for hid, group in df_ts.groupby('house_id', sort=False):
    # Time is the index
    group = group.set_index('timestamp', drop=False)
    
    # Features
    feat_ts = TimeSeries.from_dataframe(
        group,
        time_col='timestamp',
        value_cols=FEATURES,
        freq='15min',
    ).with_static_covariates(pd.DataFrame({'house_id':[hid]}))
    
    # Targets
    targ_ts = TimeSeries.from_dataframe(
        group,
        time_col='timestamp',
        value_cols=TARGETS,
        freq='15min',
    ).with_static_covariates(pd.DataFrame({'house_id':[hid]}))
    
    # Store
    house_feature_series[hid] = feat_ts
    house_target_series[hid]  = targ_ts

print(f"Built {len(house_feature_series)} house TimeSeries objects")

### Cross validation

In [ ]:
# Core libraries
import time, gc, json, warnings
import numpy as np
from sklearn.model_selection import StratifiedKFold, TimeSeriesSplit # Chane to stratifiedgroupkfold
from sklearn.metrics import roc_auc_score

# Darts
from darts.models import TCNModel

In [ ]:
# Convert the dicts built earlier into reproducible ordered lists
house_ids = np.array(sorted(house_feature_series.keys()))
feat_series_list = [house_feature_series[h] for h in house_ids]
targ_series_list = [house_target_series[h]  for h in house_ids]

print(f"Total houses: {len(house_ids)}")

In [ ]:
# CV configuration
outer_cv = StratifiedKFold(n_splits=5) 
# inner_cv = TimeSeriesSplit(n_splits=6, test_size=96, gap=96) # Not used

# TCN hyper-parameters
INPUT_CHUNK  = 7 * 96 # 1 week history
OUTPUT_CHUNK = 24 # 6h forecast horizon
N_EPOCHS = 20
BATCH_SIZE = 32
NUM_LAYERS = 3
NUM_FILTERS = 64
KERNEL_SIZE = 3
DILATION = 2
DROPOUT = 0.1

## Model training

In [ ]:
from darts.utils.likelihood_models.torch import BernoulliLikelihood

from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    log_loss,
    brier_score_loss,
    precision_score, recall_score, f1_score
)

def run_outer_fold(train_idx, test_idx, fold_no):
    """
    Train on `train_idx` houses, validate on the last OUTPUT_CHUNK of each
    remaining house, and evaluate on `test_idx`.

    Houses whose time-series are shorter than the minimum length required by
    Darts (INPUT_CHUNK + OUTPUT_CHUNK) are skipped (“dropped out”).
    """
    # Gather per-house series lists
    train_feats_outer = [feat_series_list[i] for i in train_idx]
    train_targ_outer = [targ_series_list[i] for i in train_idx]
    test_feats_outer = [feat_series_list[i] for i in test_idx]
    test_targ_outer = [targ_series_list[i] for i in test_idx]

    # Minimum length of the portion fed to model.fit()
    MIN_TRAIN_LEN = INPUT_CHUNK + OUTPUT_CHUNK # 7 days + 24 h = 768

    # Build training / validation sets, dropping houses with too few points
    train_feats, train_targs, val_feats, val_targs = [], [], [], []
    dropped = 0

    for feats_ts, targ_ts in zip(train_feats_outer, train_targ_outer):
        n_points = len(targ_ts)

        # Need MIN_TRAIN_LEN for fitting + OUTPUT_CHUNK for validation chunk
        if n_points < MIN_TRAIN_LEN + OUTPUT_CHUNK:
            dropped += 1
            continue
        
        VAL_WINDOW = INPUT_CHUNK + OUTPUT_CHUNK 
        split_idx = n_points - VAL_WINDOW
        train_feats.append(feats_ts[:split_idx])
        train_targs.append(targ_ts[:split_idx])
        val_feats.append(feats_ts[split_idx:])
        val_targs.append(targ_ts[split_idx:])

    if dropped:
        print(f"Dropped {dropped} train houses that were < "
              f"{MIN_TRAIN_LEN + OUTPUT_CHUNK} points long")

    # Define and fit the TCN
    model = TCNModel(
        input_chunk_length = INPUT_CHUNK,
        output_chunk_length = OUTPUT_CHUNK,
        output_chunk_shift = 0,
        likelihood = BernoulliLikelihood(), 
        n_epochs = N_EPOCHS,
        batch_size = BATCH_SIZE,
        num_layers = NUM_LAYERS,
        num_filters = NUM_FILTERS,
        kernel_size = KERNEL_SIZE,
        dilation_base = DILATION,
        dropout = DROPOUT,
        random_state = 42,
    )

    t0 = time.time()
    model.fit(series=train_targs,
            past_covariates=train_feats,
            val_series=val_targs,
            val_past_covariates=val_feats,
            max_samples_per_ts=5,
            verbose=True)
    fit_secs = time.time() - t0

    # Evaluate
    y_true, y_pred = [], []
    for feats_ts, targ_ts in zip(test_feats_outer, test_targ_outer):
        if len(targ_ts) < INPUT_CHUNK + OUTPUT_CHUNK:
            continue  # nothing to predict safely
        hist = targ_ts[:-OUTPUT_CHUNK]
        future = targ_ts[-OUTPUT_CHUNK:] 

        preds = model.predict(n=OUTPUT_CHUNK,
                              series=hist,
                              past_covariates=feats_ts)

        y_true.extend(future.values()[:, 0]) # first dim = burst_leak flag
        y_pred.extend(preds .values()[:, 0])

    auc = roc_auc_score(y_true, y_pred)
    pr_auc = average_precision_score(y_true, y_pred)
    #nll = log_loss(y_true, y_pred)
    brier = brier_score_loss(y_true, y_pred)

    y_pred_bin = (np.asarray(y_pred) >= 0.5).astype(int)
    precision = precision_score(y_true, y_pred_bin, zero_division=0)
    recall = recall_score(y_true, y_pred_bin, zero_division=0)
    f1 = f1_score(y_true, y_pred_bin, zero_division=0)
    print(
        f"Fold {fold_no} | AUROC {auc:.4f}  PR-AUC {pr_auc:.4f}  "
        #f"LogLoss {nll:.4f}  Brier {brier:.4f}\n"
        f"           Precision {precision:.3f}  Recall {recall:.3f}  F1 {f1:.3f}  "
        f"(fit {fit_secs/60:.1f} min)"
    )
    
    total_houses = len(house_ids)  # Total houses in dataset
    train_houses = len(train_idx)  # Houses used for training
    test_houses = len(test_idx)    # Houses used for testing
    
    # Create timestamp and folder name with dataset info
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    folder_name = f"{folder_timestamp}/{fold_no}"
    folder_path = f"weights/{folder_name}"
    
    # Create the folder
    os.makedirs(folder_path, exist_ok=True)
    
    # Save the model
    model.save(f"{folder_path}/tcn_fold{fold_no}_{timestamp}.pt")
    
    print(f"Saved model to {folder_path}/tcn_fold{fold_no}_{timestamp}.pt")

    # Clean-up
    del model
    gc.collect()

    return {
        "fold": fold_no,
        "AUROC": auc,
        "PR_AUC": pr_auc,
        #"LogLoss": nll,
        "Brier": brier,
        "Precision": precision,
        "Recall": recall,
        "F1": f1,
        "fit_seconds": fit_secs,
    }

In [ ]:
ff_scaled, feature_scaler = scale_features_simple(ff_df)

In [ ]:
results = []
house_labels = np.array([int(ts.values()[:, 0].any()) for ts in house_target_series.values()])

for fold_no, (train_idx, test_idx) in enumerate(
        outer_cv.split(X=house_ids, y=house_labels, groups=house_ids), start=1):
    print(f"\n===== OUTER FOLD {fold_no}/5 "
          f"— train houses {len(train_idx)}, test houses {len(test_idx)} =====")
    fold_result = run_outer_fold(train_idx, test_idx, fold_no)
    results.append(fold_result)

In [ ]:
print("\n========== CV SUMMARY ==========")
summary_df = pd.DataFrame(results)
display(summary_df)

metric_cols = ["AUROC", "PR_AUC", #"LogLoss", 
                "Brier",
               "Precision", "Recall", "F1"]

mean_vals = summary_df[metric_cols].mean()
print("Mean metrics across folds:")
for m, v in mean_vals.items():
    print(f" • {m:10s}: {v:.4f}")

# Save with all columns
summary_df.to_json("cv_results_250.json", orient="records", indent=2)
print("Saved per-fold metrics to cv_results_250.json")